In [64]:
import csv,os,re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sqlite3

In [65]:
connectionState = sqlite3.connect('Data/gutenberg/gutenberg.sqlite3')
#cursor=connectionState.cursor()
data_df = pd.read_sql_query("SELECT * FROM articles", connectionState)
# dropping the id column
data_df = data_df.drop('id',axis=1)
# rename columns to author and text
data_df.columns = ['Author','Text']

In [66]:
data_df['Text'] = data_df['Text'].map(lambda x: re.sub('\r|\n|\'','',x))
data_df['Text'] = data_df['Text'].map(lambda x: re.sub(r'--\d\d\d-\d\d\d-\d\d\d\d','',x))
data_df['Text'] = data_df['Text'].map(lambda x: re.sub('\s+',' ',x))

In [67]:
#Generate corpus of words for use by the CountVectorizer
corpus = []
for text in data_df['Text']:
    corpus.append(text)
#corpus[11]

In [68]:
# remove stop words
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
stop_words_subset = set([x for x in stop_words if 3 <= len(x) <=5])

def remove_stopwords(x): #x is the string
    word_set = x.split(' ')
    for word in word_set:
        if word in stop_words:
            word_set.remove(word)
    return ' '.join(word_set)


In [69]:
# removes the stopwords from the dataframe
data_df['Text'] = data_df['Text'].map(lambda x: remove_stopwords(x))

0        With mind thus full happiness, Catherine hard...
1       Mrs. John Dashwood much confidence husbands ju...
2        Everything now regular train: theatre, actors...
3        Mrs. Westons friends made happy safety; satis...
4       From night sleep expected, Marianne awoke then...
5       Mrs. Dashwood remained Norland several months;...
6       Elizabeth related Jane next day, passed Mr.Wic...
7        Emmas pensive meditations, walked home, inter...
8       Mrs. Jennings came immediately their room retu...
9        Mr. Woodhouse fond society way. He liked much...
10      After week spent professions love schemes feli...
11      Instead receiving such letter excuse friend, a...
12       Catherine much engaged theatre evening, retur...
13       Mr. Knightley might quarrel her, Emma could q...
14      "If open weather holds much longer," said Mrs....
15       Mr. Mrs. John Knightley detained long Hartfie...
16       Mr. Bertram set for————, Miss Crawford prepar...
17      Mrs. P

In [74]:
from nltk.stem import PorterStemmer 
from nltk.tokenize import word_tokenize 

porter_stemmer = PorterStemmer()
def stem_sentences(sentence):
    tokens = sentence.split()
    stemmed_tokens = [porter_stemmer.stem(token) for token in tokens]
    return ' '.join(stemmed_tokens)


In [76]:
# stem the words
data_df['Text'] = data_df['Text'].apply(stem_sentences)
#data_df['Text']

0       with mind thu full happiness, catherin hardli ...
1       mrs. john dashwood much confid husband judgmen...
2       everyth now regular train: theatre, actors, ac...
3       mrs. weston friend made happi safety; satisfac...
4       from night sleep expected, mariann awok thenex...
5       mrs. dashwood remain norland sever months; any...
6       elizabeth relat jane next day, pass mr.wickham...
7       emma pensiv meditations, walk home, interrupte...
8       mrs. jen came immedi their room return, andwit...
9       mr. woodhou fond societi way. He like much fri...
10      after week spent profess love scheme felicity,...
11      instead receiv such letter excu friend, aseliz...
12      catherin much engag theatr evening, return nod...
13      mr. knightley might quarrel her, emma could qu...
14      "if open weather hold much longer," said mrs. ...
15      mr. mrs. john knightley detain long hartfield....
16      mr. bertram set for————, miss crawford prepar ...
17      mrs. p

In [77]:
from sklearn.model_selection import train_test_split
y = data_df.pop("Author")
#X = data_df
#print(y)
X_train, X_test, y_train, y_test = train_test_split(data_df['Text'], y, test_size=0.3, stratify=y,random_state=42) 


In [78]:
# Both test and train labels contain instances of all the 42 authors. 
df_ytrain = pd.DataFrame(y_train)
df_ytest = pd.DataFrame(y_test)
print(len(df_ytrain['Author'].value_counts()))
print(len(df_ytest['Author'].value_counts()))
type(df_ytrain)

42
42


pandas.core.frame.DataFrame

In [79]:
from sklearn import preprocessing, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# label encode the target variable 
encoder = preprocessing.LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.fit_transform(y_test)

# create a count vectorizer object 
count_vect = CountVectorizer(analyzer='word',max_df=0.50,min_df=0.01)
#count_vect.fit(corpus)

# transform the training and validation data using count vectorizer object
Xtrain_count =  count_vect.fit_transform(X_train)
Xtest_count =  count_vect.transform(X_test)

#print(count_vect.get_feature_names())

print(Xtrain_count.__str__)
print(Xtrain_count)

<bound method spmatrix.__str__ of <1470x5278 sparse matrix of type '<class 'numpy.int64'>'
	with 394418 stored elements in Compressed Sparse Row format>>
  (0, 242)	1
  (0, 3680)	1
  (0, 1461)	1
  (0, 1090)	1
  (0, 3746)	1
  (0, 3811)	1
  (0, 2580)	1
  (0, 340)	1
  (0, 5275)	1
  (0, 454)	1
  (0, 4069)	1
  (0, 5276)	1
  (0, 3152)	1
  (0, 4996)	1
  (0, 4614)	1
  (0, 3835)	1
  (0, 1632)	1
  (0, 1256)	1
  (0, 4497)	1
  (0, 3833)	1
  (0, 5260)	2
  (0, 4228)	2
  (0, 3174)	1
  (0, 357)	1
  (0, 1333)	1
  :	:
  (1469, 3739)	1
  (1469, 4164)	2
  (1469, 3252)	1
  (1469, 2397)	1
  (1469, 2867)	3
  (1469, 2234)	4
  (1469, 1395)	1
  (1469, 4232)	1
  (1469, 715)	1
  (1469, 2242)	2
  (1469, 4589)	1
  (1469, 2699)	1
  (1469, 1715)	2
  (1469, 670)	1
  (1469, 4279)	2
  (1469, 4885)	2
  (1469, 2177)	2
  (1469, 2028)	1
  (1469, 4790)	1
  (1469, 3716)	1
  (1469, 3557)	3
  (1469, 3415)	2
  (1469, 3998)	2
  (1469, 2024)	1
  (1469, 4101)	1


In [80]:
# word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer='word',sublinear_tf=True, max_df=0.50,min_df=0.01)
#tfidf_vect.fit(corpus)
Xtrain_tfidf = tfidf_vect.fit_transform(X_train)
Xtest_tfidf = tfidf_vect.transform(X_test)

#print(Xtrain_tfidf.__str__)
#print(Xtrain_tfidf)

# ngram level tf-idf 
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', ngram_range=(2,3),sublinear_tf=True, max_df=0.50,min_df=0.01)
#tfidf_vect_ngram.fit(corpus)
Xtrain_tfidf_ngram = tfidf_vect_ngram.fit_transform(X_train)
Xtest_tfidf_ngram = tfidf_vect_ngram.transform(X_test)

print(Xtrain_tfidf_ngram.__str__)
print(Xtrain_tfidf_ngram)

<bound method spmatrix.__str__ of <1470x3757 sparse matrix of type '<class 'numpy.float64'>'
	with 130085 stored elements in Compressed Sparse Row format>>
  (0, 2891)	0.09323585187294296
  (0, 3326)	0.09165144279181674
  (0, 3211)	0.08838376403404236
  (0, 3751)	0.1401478327378658
  (0, 1118)	0.06298050449166656
  (0, 2810)	0.05159888646890965
  (0, 172)	0.1048708222119551
  (0, 1096)	0.09751589019955616
  (0, 506)	0.1048708222119551
  (0, 1617)	0.08795805845956782
  (0, 3745)	0.10808781068267771
  (0, 2263)	0.09379348160070491
  (0, 127)	0.08123766865943251
  (0, 2245)	0.21250936219319042
  (0, 1664)	0.10928784756470787
  (0, 845)	0.06115903214265201
  (0, 2248)	0.09115028827005658
  (0, 3204)	0.10390504304947375
  (0, 146)	0.0803577661881599
  (0, 2096)	0.06298050449166656
  (0, 3667)	0.07844508181757766
  (0, 2234)	0.09893038796895902
  (0, 3044)	0.08634127889489675
  (0, 853)	0.09379348160070491
  (0, 1334)	0.09066150920745308
  :	:
  (1469, 2229)	0.07615846838446338
  (1469, 216)

In [81]:
def train_model(classifier, feature_vector_train, label, feature_vector_test, is_neural_net=False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_test)
    
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)
    
    return metrics.accuracy_score(predictions, y_test)

In [84]:
#print(Xtrain_tfidf_ngram.shape)
# Naive Bayes on Count Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), Xtrain_count, y_train, Xtest_count)
print ("NB, Count Vectors: ", accuracy)


NB, Count Vectors:  0.8714285714285714


In [85]:
# Naive Bayes on Word Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), Xtrain_tfidf, y_train, Xtest_tfidf)
print ("NB, WordLevel TF-IDF: ", accuracy)

# Naive Bayes on Ngram Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), Xtrain_tfidf_ngram, y_train, Xtest_tfidf_ngram)
print ("NB, N-Gram Vectors: ", accuracy)

NB, WordLevel TF-IDF:  0.8
NB, N-Gram Vectors:  0.5206349206349207


In [86]:
# SVM on Ngram Level TF IDF Vectors

accuracy = train_model(svm.SVC(), Xtrain_tfidf_ngram, y_train, Xtest_tfidf_ngram)
print ("SVM, N-Gram Vectors: ", accuracy)

/home/ritik/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVM, N-Gram Vectors:  0.48095238095238096
